# Importing libraries

In [1]:
import numpy as np
import scipy as sp

# De-scaling Decision Matrix

In [2]:
def de_scale(D:np.ndarray, method:str, type_c):
    """
    D is the data matrix
    method is the method to be used for scaling ("Euclidean", "Linear", "Fuzzy")
    type_c is the type of criterias.In Euclidean case, it is the value of p 
    """
    Res = []
    if method == "Euclidean":
        for j in range(D.shape[1]):
            Res.append(D[:,j]/np.linalg.norm(D[:,j], type_c))
        Res = np.array(Res).transpose()
        return (Res)
    elif method == "Linear":
        for j in range(D.shape[1]):
            if type_c[j] == "+":
                Res.append(D[:,j]/np.max(D[:,j]))
            else:
                Res.append(np.min(D[:,j])/D[:,j])
        Res = np.array(Res).transpose()
        return (Res)
    elif method=="Fuzzy":
        for j in range(D.shape[1]):
            if type_c[j] == "+":
                Res.append((D[:,j]-np.min(D[:,j]))/(np.max(D[:,j])-np.min(D[:,j])))
            else:
                Res.append((np.max(D[:,j])-D[:,j])/(np.max(D[:,j])-np.min(D[:,j])))
        Res = np.array(Res).transpose()
        return (Res)
    else:
        print("Invalid method")

In [36]:
print(de_scale(np.array([[1,2,3],[4,5,6],[7,8,9]]), "Fuzzy", ("+","-","+")))
print(de_scale(np.array([[1,2,3],[4,5,6],[7,8,9]]), "Euclidean", 2))

[[0.  1.  0. ]
 [0.5 0.5 0.5]
 [1.  0.  1. ]]
[[0.12309149 0.20739034 0.26726124]
 [0.49236596 0.51847585 0.53452248]
 [0.86164044 0.82956136 0.80178373]]


# Checking whether a matrix is inconsistent or not

In [4]:
def is_consistent(D:np.ndarray):
    """
    D is the data matrix
    """
    
    return (f"lambda_max = {np.max(np.linalg.eig(D)[0])},{np.max(np.linalg.eig(D)[0])==D.shape[1]}")

In [5]:
is_consistent(np.array([[1,1/3,1/2],[3,1,3],[2,1/3,1]]))

'lambda_max = (3.0536215758789735+0j),False'

# Approximate methods of finding weights

In [6]:
def app_weights(D:np.ndarray, method:str):
    """
    D is the data matrix
    method is the method to be used ("Row", "Column", "AHP", "Geometric")
    """
    if method == "Row":
        Res = np.sum(D, axis=1)
        Res = Res/np.sum(Res)
        Res = Res.reshape(Res.shape[0],1)
        return(Res)
    elif method == "Column":
        Res = np.sum(D, axis=0)
        Res = 1/Res
        Res = Res/np.sum(Res)
        Res = Res.reshape(Res.shape[0],1)
        return(Res)
    elif method == "AHP":
        new_mat = []
        c_sum = np.sum(D, axis=0)
        for j in range(D.shape[1]):
            new_mat.append(D[:,j]/c_sum[j])
        new_mat = np.array(new_mat).transpose()
        return(np.mean(new_mat,axis=1).reshape(new_mat.shape[0],1))
    elif method == "Geometric":
        Res = np.prod(D, axis=1)
        Res = Res**(1/D.shape[1])
        Res = Res/np.sum(Res)
        Res = Res.reshape(Res.shape[0],1)
        return(Res)
    else:
        print("Invalid method")

In [7]:
app_weights(np.array([[1,6,1/3,4],[1/6,1,1/7,1/2],[3,7,1,5],[1/4,2,1/5,1]]), "Geometric")

array([[0.29119249],
       [0.05719234],
       [0.5542491 ],
       [0.09736608]])

# Least Square Method

In [8]:
from scipy.optimize import minimize
def least_s(D:np.ndarray):
    def f(w):
        res = 0
        for i in range(D.shape[0]):
            for j in range(D.shape[1]):
                res += (D[i,j]*w[j] - w[i])**2
        return res
    def cons(w):
        res = 0
        for i in range(len(w)):
            res += w[i]
        return res - 1
    x0 = np.zeros(D.shape[0])
    return minimize(f,x0 , constraints={"type":"eq", "fun":cons})

In [32]:
least_s(np.array([[1,2,4],[1/2,1,3],[1/4,1/3,1]]))

     fun: 0.012433263382089669
     jac: array([0.02390231, 0.02928199, 0.01863343])
 message: 'Optimization terminated successfully'
    nfev: 26
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.56842712, 0.30409565, 0.12747723])

# Printing Lagrangian Equations

In [41]:
def print_lagrangian(D:np.ndarray):
    for k in range(D.shape[1]):
        print(f"k = {k+1} ", end=": ")
        for j in range(D.shape[0]):
            print(f"({D[j,k]}* W{k+1} - W{j+1}) *{D[j,k]} - ({D[k,j]}* W{j+1} - W{k+1}) ", end=" + ")
        print("lambda = 0")
    for i in range(D.shape[0]-1):
        print(f" W{i+1} +", end="")
    print(f" W{D.shape[0]} = 1")

In [42]:
print_lagrangian(np.array([[1,1/3,5],[3,1,7],[1/5,1/7,1]]))

k = 1 : (1.0* W1 - W1) *1.0 - (1.0* W1 - W1)  + (3.0* W1 - W2) *3.0 - (0.3333333333333333* W2 - W1)  + (0.2* W1 - W3) *0.2 - (5.0* W3 - W1)  + lambda = 0
k = 2 : (0.3333333333333333* W2 - W1) *0.3333333333333333 - (3.0* W1 - W2)  + (1.0* W2 - W2) *1.0 - (1.0* W2 - W2)  + (0.14285714285714285* W2 - W3) *0.14285714285714285 - (7.0* W3 - W2)  + lambda = 0
k = 3 : (5.0* W3 - W1) *5.0 - (0.2* W1 - W3)  + (7.0* W3 - W2) *7.0 - (0.14285714285714285* W2 - W3)  + (1.0* W3 - W3) *1.0 - (1.0* W3 - W3)  + lambda = 0
 W1 + W2 + W3 = 1


# Logarithmic Least Squares

Couldnt code it :(

# Eigen Value Method

In [12]:
def eigen_method(D:np.ndarray):
    """
    D is the data matrix
    """
    Lam_max = np.real(max(np.linalg.eig(D)[0]))
    w_u = sp.linalg.null_space(D - np.eye(D.shape[0])*Lam_max)
    w_u = w_u/np.sum(w_u)
    print(w_u)

In [13]:
eigen_method(np.array([[1,1/3,5],[3,1,7],[1/5,1/7,1]]))

[[0.27895457]
 [0.649118  ]
 [0.07192743]]


# Sa'ati Method

In [47]:
def saati(D:np.ndarray):
    """
    D is the data matrix
    """
    w_init = app_weights(D, "Row")
    print(f"D: {D}")
    print(f"w_init: {w_init}")
    D_new = D
    while True:
        D_new = np.matmul(D_new,D_new)
        print(f"D_new: {D_new}")
        w_new = app_weights(D_new, "Row")
        print(f"w_new: {w_new}")
        if np.linalg.norm(w_new-w_init) < 0.0001:
            break
        w_init = w_new

In [48]:
saati(np.array([[1,1/9,1/3,1/4],[9,1,3,2],[3,1/3,1,1/2],[4,1/2,2,1]]))

D: [[1.         0.11111111 0.33333333 0.25      ]
 [9.         1.         3.         2.        ]
 [3.         0.33333333 1.         0.5       ]
 [4.         0.5        2.         1.        ]]
w_init: [[0.05837321]
 [0.51674641]
 [0.16650718]
 [0.25837321]]
D_new: [[ 4.          0.45833333  1.5         0.88888889]
 [35.          4.         13.          7.75      ]
 [11.          1.25        4.          2.41666667]
 [18.5         2.11111111  6.83333333  4.        ]]
w_new: [[0.05866952]
 [0.51196001]
 [0.15994288]
 [0.26942759]]
D_new: [[ 64.98611111   7.41820988  24.03240741  14.28819444]
 [566.375       64.65277778 209.45833333 124.52777778]
 [176.45833333  20.14351852  65.26388889  38.79861111]
 [297.05555556  33.90972222 109.86111111  65.31944444]]
w_new: [[0.05881649]
 [0.51261024]
 [0.1597113 ]
 [0.26886197]]
D_new: [[ 16909.78935185   1930.29443426   6253.74385931   3718.02833505]
 [147376.5623071   16823.40046296  54504.24324846  32404.32055363]
 [ 45917.82619599   5241.63385738 

# Entropy Method

In [16]:
def Entropy(D:np.ndarray,lam:np.ndarray):
    """
    D is the data matrix
    lam is the weights in our oponion
    """
    P = []
    for j in range(D.shape[1]):
        P.append(D[:,j]/np.sum(D[:,j]))
    P = np.array(P).transpose()
    print(f"P: {P},{P.shape}")
    E = []
    for j in range(P.shape[1]):
        E.append(
        -1 * (1/np.log(P.shape[0])) * np.sum(P[:,j]*np.log(P[:,j]))
        )
        print(f"Entropy of criteria {j+1} is {E[j]}")
    D = 1 - np.array(E)
    print(f"D: {D}")
    D = D/np.sum(D)
    print(f"Weight of criteria: {D}")
    new_w = np.multiply(D,lam)/np.sum(np.multiply(D,lam))
    print(f"New weights: {new_w}")

In [17]:
Entropy(np.array([[80,12,9,7,260],[75,14,9,9,230],[72,13,7,7,50],[65,15,5,7,140]]),np.array([0.3,0.1,0.3,0.2,0.1]))

P: [[0.2739726  0.22222222 0.3        0.23333333 0.38235294]
 [0.25684932 0.25925926 0.3        0.3        0.33823529]
 [0.24657534 0.24074074 0.23333333 0.23333333 0.07352941]
 [0.22260274 0.27777778 0.16666667 0.23333333 0.20588235]],(4, 5)
Entropy of criteria 1 is 0.9979873525070015
Entropy of criteria 2 is 0.997521582679676
Entropy of criteria 3 is 0.9814490485575648
Entropy of criteria 4 is 0.9953823248677508
Entropy of criteria 5 is 0.9028051923588244
D: [0.00201265 0.00247842 0.01855095 0.00461768 0.09719481]
Weight of criteria: [0.01611994 0.01985044 0.14858056 0.03698445 0.7784646 ]
New weights: [0.03539252 0.0145277  0.32621957 0.05413473 0.56972547]


# Inconsistancy Assessment Algorithms

In [30]:
def IAA(D:np.ndarray):
    """
    D is the data matrix
    """
    w = app_weights(D, "AHP")
    print(f"W: {w}")
    w_prime = np.matmul(D,w)
    print(f"W': {w_prime}")
    lambda_max = np.divide(w_prime, w)
    print(f"lambda_max: {lambda_max}")
    lambda_max = np.mean(lambda_max)
    print(f"lambda_max_avg: {lambda_max}")
    CI = (lambda_max - D.shape[0])/(D.shape[0]-1)
    print(f"CI: {CI}")
    RI = [0,0,0.58,0.9,1.12,1.24,1.32,1.41,1.45,1.49]
    CR = CI/RI[D.shape[0]-1]
    print(f"CR: {CR}")
    if CR < 0.1:
        print("Consistency is good")

In [33]:
IAA(np.array([[1,1/2,1/6],[2,1,1/5],[6,5,1]]))

W: [[0.10332847]
 [0.17413661]
 [0.72253492]]
W': [[0.31081926]
 [0.52530054]
 [2.2131888 ]]
lambda_max: [[3.00806994]
 [3.01660016]
 [3.06308906]]
lambda_max_avg: 3.0292530515599956
CI: 0.01462652577999779
CR: 0.025218147896547916
Consistency is good
